In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
%load_ext line_profiler

# Synthetic data


In [4]:
np.random.seed(0)
n_feat = 10
size1, size2 = 10000, 10000
s1 = pd.DataFrame(np.random.normal(size=(size1, n_feat)))
s2 = pd.DataFrame(s1.values[:size2] + np.random.normal(size=(size2, n_feat)))

In [5]:
s1.head()

,0,1,2,3,4,5,6,7,8,9
0,1.764052,0.400157,0.978738,2.240893,1.867558,-0.977278,0.950088,-0.151357,-0.103219,0.410599
1,0.144044,1.454274,0.761038,0.121675,0.443863,0.333674,1.494079,-0.205158,0.313068,-0.854096
2,-2.552990,0.653619,0.864436,-0.742165,2.269755,-1.454366,0.045759,-0.187184,1.532779,1.469359
3,0.154947,0.378163,-0.887786,-1.980796,-0.347912,0.156349,1.230291,1.202380,-0.387327,-0.302303
4,-1.048553,-1.420018,-1.706270,1.950775,-0.509652,-0.438074,-1.252795,0.777490,-1.613898,-0.212740


In [6]:
s2.head()

,0,1,2,3,4,5,6,7,8,9
0,1.280255,1.688214,0.848859,2.042815,1.533070,-1.368721,0.337682,-0.827881,1.224011,-0.038097
1,-0.172364,1.485104,0.447681,-0.051584,0.116495,1.278043,2.616096,-0.092820,1.685408,1.208466
2,-5.001549,0.091630,1.219535,-0.107787,1.877305,-2.267522,-0.187115,0.059617,0.970366,0.627757
3,-0.787173,1.323536,-0.746875,-0.227231,-0.695136,-1.333144,0.722255,3.195030,0.168135,-1.998380
4,0.703813,-2.017569,-2.172762,0.855983,-1.155077,-0.640229,-1.047722,0.507105,-1.924197,0.149472


# Testing

In [7]:
import itertools
import multiprocessing

In [8]:
from big_roc import calc_gen_imp_hist
from big_roc.big_roc import _calc_gen_imp_hist

In [9]:
sim_min = -1
sim_max = 1
eps = 1e-4
n_intervals = 1000000
intervals = np.linspace(sim_min - eps, sim_max + eps, n_intervals)

In [17]:
%%time
results = calc_gen_imp_hist(s1, s2, intervals, batch_size=1000)

TypeError: 'numpy.float64' object cannot be interpreted as an integer

In [14]:
%%time
results = calc_gen_imp_hist(s1, s2, intervals, batch_size=1000, n_workers=1)

Wall time: 35.1 s


In [28]:
%lprun -f calc_gen_imp_hist calc_gen_imp_hist(s1, s2, intervals, batch_size=10000)

Timer unit: 1e-07 s

Total time: 0.104059 s
File: C:\Users\v_p164\Projects\bigROC\big_roc\big_roc.py
Function: calc_gen_imp_hist at line 46

Line #      Hits         Time  Per Hit   % Time  Line Contents
    46                                           def calc_gen_imp_hist(s1: pd.DataFrame, s2: pd.DataFrame, bin_edges: np.array, batch_size: int = 10000,
    47                                                                 similarity_measure: Callable = cosine_similarity, n_workers: int = multiprocessing.cpu_count()) \
    48                                                   -> Tuple[np.array, np.array]:
    49         1     910920.0 910920.0     87.5      p = multiprocessing.Pool(n_workers)
    50         1         33.0     33.0      0.0      func = partial(_calc_gen_imp_hist,
    51         1      16378.0  16378.0      1.6                     s2_batches=split(s2, batch_size), bin_edges=bin_edges, similarity_measure=similarity_measure
    52                                           

In [12]:
%%time
calc_gen_imp_hist_c(s1, s2, intervals, batch_size=10000)

Wall time: 13.6 s


(array([[-0.998, -0.996, -0.994, ...,  0.996,  0.998,  1.   ],
        [ 0.   ,  0.   ,  0.   , ...,  0.   ,  0.   ,  0.   ]]),
 array([[-0.998, -0.996, -0.994, ...,  0.996,  0.998,  1.   ],
        [ 0.   ,  0.   ,  0.   , ...,  0.   ,  0.   ,  0.   ]]))

In [ ]:
%%time
calc_gen_imp_hist(s1, s2, intervals, batch_size=10000)

In [ ]:
calc_gen_imp_hist calc_gen_imp_hist(s1, s2, intervals, batch_size=10000)

In [18]:
gen_bins

array([[-0.998, -0.996, -0.994, ...,  0.996,  0.998,  1.   ],
       [ 0.   ,  0.   ,  0.   , ...,  0.   ,  0.   ,  0.   ]])